In [1]:
machine = 'mac'
machine = 'magny'

In [2]:
if machine == 'magny':
    filedir = "/home/extmilan/masterthesis/files/"
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac':
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
else:
    raise NotADirectoryError

In [3]:
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, HernquistPotential
from galpy.actionAngle import estimateDeltaStaeckel, actionAngleStaeckel

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *
from auriga_functions import *

import numpy as np
from IPython.display import Image, display

import corner

import datetime
import random

import copy

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Circle
from matplotlib import animation

%matplotlib inline

In [4]:
#### path = /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/*
level = 4
if machine == 'magny':
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
elif machine == 'mac': 
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
j = 0
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"

    for snapnr in range(127,128,1):
        print("level   : {0}".format(level))
        print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print("basedir : {0}".format(basedir))
        print("halodir : {0}".format(halodir))
        print("snappath: {0}\n".format(snappath))
        s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
            haloid=0, galradfac=0.1, verbose=True) 

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )
        

level   : 4
halo    : 24
snapnr  : 127
basedir : /hits/universe/GigaGalaxy/level4_MHD/
halodir : /hits/universe/GigaGalaxy/level4_MHD/halo_24/
snappath: /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/

[ 34.42281723  33.16259384  37.29567337]
Found 1783163 stars.
Rotated pos.
Rotated vel.

galrad  : 0.02408556640148163
redshift: 2.220446049250313e-16
time    : 0.9999999999999998
center  : [ 0.  0.  0.]



In [5]:
#_____reference values for galpy_____
_REFR0_kpc = 8.

#_____function that sets-up galpy potential_____
def setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB):
    
    #test input:
    if (a_MND_kpc <= 0.) or (b_MND_kpc <= 0.) or (a_NFWH_kpc <= 0.) or (a_HB_kpc <= 0.) \
       or (n_MND <= 0.) or (n_NFWH <= 0.) or (n_HB <= 0.) or (n_MND >= 1.) or (n_NFWH >= 1.) or (n_HB >= 1.):
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The input parameters for the scaling profiles do not correspond to a physical potential.')
    if np.fabs(n_MND + n_NFWH + n_HB - 1.) > 1e-7:
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The sum of the normalization does not add up to 1.')
        
    #trafo to galpy units:
    a_MND  = a_MND_kpc / _REFR0_kpc
    b_MND  = b_MND_kpc / _REFR0_kpc
    a_NFWH = a_NFWH_kpc / _REFR0_kpc
    a_HB   = a_HB_kpc / _REFR0_kpc
    
    #setup potential:
    disk = MiyamotoNagaiPotential(
                a = a_MND,
                b = b_MND,
                normalize = n_MND)
    halo = NFWPotential(
                a = a_NFWH,
                normalize = n_NFWH)
    bulge = HernquistPotential(
                a = a_HB,
                normalize = n_HB)
    return [disk, halo, bulge]


Action evolution parameters
---

- IDlist (somehow.... find where dwarf (67 für den Anfangs) merges)
- startnr
- fit routine = emcxee 1 (in v0.1; more maybe in v0.2)
- delta fix or calculation
- figure (with labels, range, title etc predefined)
- in v0.2 possibility for wrong potential


Get IDs of merged stars from snapshot 72 to 73
---
Maybe instead take table made in 2dwarf

In [6]:
level = 4
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"
    for snapnr in range(73,74,1):
        print("snapnr  : {0}".format(snapnr))
        print(datetime.datetime.now().time())
        #print("snappath: {0}\n".format(snappath))
        s1, sf1 = eat_snap_and_fof(level, halo_number, snapnr-1, snappath, loadonlytype=[4], 
            haloid=0, galradfac=0.1, verbose=False) 
        
        s2, sf2 = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
            haloid=0, galradfac=0.1, verbose=False) 
        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s1.data['gmet'] = np.maximum( s1.data['gmet'], 1e-40 )
        s2.data['gmet'] = np.maximum( s2.data['gmet'], 1e-40 )
        
        # get positions and velocities of all stars 
        istars1, = np.where( (s1.type == 4) & (s1.halo == 0) )#& (s.subhalo == 0))
        (x1_kpc, y1_kpc, z1_kpc), (vx1_kms, vy1_kms, vz1_kms), rxyz1_kpc, rxy1_kpc = get_cartesian_vectors(s1, sf1, istars1)
        istars2, = np.where( (s2.type == 4) & (s2.halo == 0) )#& (s.subhalo == 0))
        (x2_kpc, y2_kpc, z2_kpc), (vx2_kms, vy2_kms, vz2_kms), rxyz2_kpc, rxy2_kpc = get_cartesian_vectors(s2, sf2, istars2)
 
        # mask for stars in first snapshot in first subhalo
        istarssub1, = np.where( (s1.type == 4) & (s1.halo == 0) & (s1.subhalo == 1) )
        # get their IDs
        ids_halo1 = s1.id[istarssub1]
        # check which of these stars are in the main galaxy in the snapshot after
        idsmergermask = np.isin(s2.id[np.where(s2.subhalo ==0)], ids_halo1)
        # get their IDs
        ids_merger = s2.id[np.where(s2.subhalo ==0)][idsmergermask]        

snapnr  : 73
23:05:45.267365
[ 15.1824646   14.63997269  15.67546272]
Found 536367 stars.
Rotated pos.
Rotated vel.
[ 15.47813797  14.92653275  15.98590374]
Found 667800 stars.
Rotated pos.
Rotated vel.


In [7]:
ids_merger = np.loadtxt(filedir + 'surviving_ids_snapshot_72_sh_1.txt')

In [8]:
part_num = 1000
rand_samp = np.array(random.sample(list(enumerate(ids_merger)), part_num))
rand_ind = rand_samp[:,0].astype(int)
IDS = rand_samp[:, 1]
IDS = ids_merger


In [ ]:
# load fitted potential parameters
pot_params = np.loadtxt(filedir + 'potential_params_1000.txt')

snapnr          = pot_params[:,0]
v0_kms          = pot_params[:,1]
v0_kms_err      = pot_params[:,2]
a_MND_kpc       = pot_params[:,3]
a_MND_kpc_err   = pot_params[:,4]
b_MND_kpc       = pot_params[:,5]
b_MND_kpc_err   = pot_params[:,6]
a_NFWH_kpc      = pot_params[:,7]
a_NFWH_kpc_err  = pot_params[:,8]
a_HB_kpc        = pot_params[:,9]
a_HB_kpc_err    = pot_params[:,10]
n_MND           = pot_params[:,11]
n_MND_err       = pot_params[:,12]
n_NFWH          = pot_params[:,13]
n_NFWH_err      = pot_params[:,14]
n_HB            = pot_params[:,15]
n_HB_err        = pot_params[:,16]

# make snapnrs to integers
snapnr = np.array(snapnr, dtype = int)

# load smoothed fitted potential parameters
jj = 17
fitted_pot_params = np.loadtxt(filedir + 'fitted_potparams_w_' + str(jj)+'_pols.txt')
#v0_kms          = fitted_pot_params[0, :]
#a_MND_kpc       = fitted_pot_params[1, :]
#b_MND_kpc       = fitted_pot_params[2, :]
#a_NFWH_kpc      = fitted_pot_params[3, :]
#a_HB_kpc        = fitted_pot_params[4, :]
#n_MND           = fitted_pot_params[5, :]
#n_NFWH          = fitted_pot_params[6, :]
#n_HB            = fitted_pot_params[7, :]


# startnr is number of first snapshot to calculate its actions (of the GCs with merger IDs)
startnr = 127
# endnumber is last snapshot plus 1 (since I use range function)
endnr = int(np.max(snapnr) + 1) 
# NRSNAPSHOTS USELESS ATM
nrsnapshots = int(endnr-startnr)
# helping parameter to distinguish between first corner plot as base figure to plot other corner plots on
init = 0

# figure related color 
color = 'pink'


halo_number = 24  # range(1, 31):
halodir = basedir + "halo_{0}/".format(halo_number)
snappath = halodir + "output/"
for snap_number in range(startnr, endnr, 1):
    # make index starting from 0 for potential parameters
    ii = int(snap_number - startnr)
    print("snapnr: {0}".format(snap_number))
    print(datetime.datetime.now().time())
    s1, sf1 = eat_snap_and_fof(level, halo_number, snap_number, snappath, loadonlytype=[4], 
        haloid=0, galradfac=0.1, verbose=False) 

    # Clean negative and zero values of gmet to avoid RuntimeErrors
    # later on (e.g. dividing by zero)
    s1.data['gmet'] = np.maximum( s1.data['gmet'], 1e-40 ) 

    # create masks for merged stars according to their IDs to get their positions
    gcmask = np.isin(s1.id, IDS)
    # PROBABLY DO NOT NEED THESE VALUES
    (x1_merg_kpc, y1_merg_kpc, z1_merg_kpc), (vx1_merg_kms, vy1_merg_kms, vz1_merg_kms), rxyz1_merg_kpc, rxy1_merg_kpc = get_cartesian_vectors(s1, sf1, gcmask)
    
    # set up galpy potential with values of potential fitting (either smoothed or not, depends on commenting out or not)
    pot_galpy = setup_galpy_potential(a_MND_kpc[ii], b_MND_kpc[ii], a_NFWH_kpc[ii], a_HB_kpc[ii], n_MND[ii], n_NFWH[ii], n_HB[ii])
    
    # this would mean that there are no merged GCs
    if np.sum(gcmask) == 0:
        continue
        
    # get position and velocities of all selected GCs & convert to galpy units
    (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s1, sf1, gcmask)
    # convert physical to galpy units by dividing by REF vals (get velocities from best fit parameters)
    R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / _REFR0_kpc, vR_kms / v0_kms[ii], vphi_kms / v0_kms[ii], z_kpc / _REFR0_kpc, vz_kms / v0_kms[ii]

    # estimate Delta of the Staeckel potential
    delta = 0.45
    delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)
    # CHECK HOW BIG INFLUENCE OF DELTA IS
    
    
    # set up the actionAngleStaeckel object
    aAS = actionAngleStaeckel(
            pot   = pot_galpy,  # potential
            delta = delta,      # focal length of confocal coordinate system
            c     = True        # use C code (for speed)
            )
    #### calculate actions (in galpy units) & convert them to physical units
    # CONTINUE COMMENTING FROM HERE
    r_condition = np.where((np.sqrt(R_kpc**2 + z_kpc**2)) <= (2000. * s1.galrad))
    jR_kpckms, lz_kpckms, jz_kpckms = np.zeros(len(IDS)), np.zeros(len(IDS)), np.zeros(len(IDS))
    savedids = np.zeros(len(IDS))
    for test_i, item in enumerate(IDS):
        if np.sum(np.isin(r_condition, test_i)):
            try: 
                jR_galpy, lz_galpy, jz_galpy = aAS(R_galpy[test_i], vR_galpy[test_i], vT_galpy[test_i], z_galpy[test_i], vz_galpy[test_i])
                jR_kpckms[test_i], lz_kpckms[test_i], jz_kpckms[test_i] = jR_galpy * _REFR0_kpc * v0_kms[ii], lz_galpy * _REFR0_kpc * v0_kms[ii], jz_galpy * _REFR0_kpc * v0_kms[ii]
                savedids[test_i] = item
            except ValueError:
                continue
    survivor_id_mask = np.isin(IDS, savedids)
    survivor_ids = IDS[survivor_id_mask]
    print(len(survivor_ids))
    print(2000. * s1.galrad)
    # prepare data for plotting and create corner plot
    data = np.vstack([jR_kpckms[survivor_id_mask], lz_kpckms[survivor_id_mask], jz_kpckms[survivor_id_mask]])
    
    print(data)
    labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']
    if init == 0:
        print('starting first figure')
        figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])
        init +=1
    else:
        print('start other figures')
        figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, fig = figure, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])            
    #figure.suptitle(num + ' ' + title + '\nnumber of GCs: ' + str(len(R_kpc[r_condition])) + '\n$\Delta$ Staeckel: ' + "{:2.2f}".format(delta));

    #display(figure)
    if repeater == 0:
        print('number of all selected GCs', np.sum(gcmask), 'num of GCs within 50kpc:', len(r_condition[0]))

snapnr: 127
23:06:01.540189
[ 34.42281723  33.16259384  37.29567337]
Found 1783163 stars.
Rotated pos.
Rotated vel.
6258
48.171132803
[[  2.50447149e+04   7.07998935e+04   7.20648963e+04 ...,   3.31486820e+05
    2.03981860e+06   1.60467751e+06]
 [ -1.73815750e+02   2.12997671e+02  -7.23338685e+01 ...,  -1.87551930e+03
   -1.20247424e+04  -1.12021985e+04]
 [  6.53658796e+04   1.23293820e+05   3.39522906e+04 ...,   8.59573255e+04
    3.07417798e+05   2.24638180e+05]]


/home/extmilan/anaconda3/lib/python3.6/site-packages/corner/corner.py:532: RuntimeWarning: invalid value encountered in true_divide
  sm /= sm[-1]

/home/extmilan/anaconda3/lib/python3.6/site-packages/corner/corner.py:536: RuntimeWarning: invalid value encountered in less_equal
  V[i] = Hflat[sm <= v0][-1]



In [ ]:
figure

In [ ]:
cmap = copy.copy(plt.cm.inferno)
cmap.set_bad((0,0,0))  # Fill background with black
norm=matplotlib.colors.LogNorm()
s_snap = np.arange(startnr,endnr,1)

### for xy plot ###
X = x_mergers[0]
Y = y_mergers[0]
X = np.append(X, [-151, 151])
Y = np.append(Y, [-151, 151])

fig, ax = plt.subplots(figsize = (8,8))
ax.set_ylim([-150, 150])
ax.set_xlim([-150, 150])
#Create 2d Histogram
data,x,y = np.histogram2d(X,Y, bins = 401)
#data_bg,x_bg,y_bg = np.histogram2d(X_bg,Y_bg, bins = 501)


#Smooth with filter
ext = (np.min(X), np.max(X), np.min(Y), np.max(Y))
im = plt.imshow(data.T, interpolation = 'gaussian', cmap = cmap, norm=norm, extent = ext, animated=True)
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
time_text = ax.text(0.1, 0.9,'', color = 'orange', transform=ax.transAxes, fontsize=16)

#Define animation. 
def animate(i) :
    
    X = x_mergers[i]
    Y = y_mergers[i]
    X = np.append(X, [-201, 201])
    Y = np.append(Y, [-201, 201])
    data,x,y = np.histogram2d(X,Y, bins = 501)
    im.set_data(data.T)
    time_text.set_text('Snapshot ' + str(snapnrs[i]) )
    

anim_xy = animation.FuncAnimation(fig, animate, frames=len(s_snap))

plt.show()

anim_xy.save('../plots/merger73_spatial_xy_dist.gif',writer='imagemagick',fps=3)

### for Rz plot ###
X = R_mergers[0]
Y = z_mergers[0]
X = np.append(X, [-151, 151])
Y = np.append(Y, [-151, 151])

fig, ax = plt.subplots(figsize = (8,8))
ax.set_ylim([-150, 150])
ax.set_xlim([-150, 150])
#Create 2d Histogram
data,x,y = np.histogram2d(X,Y, bins = 401)
#data_bg,x_bg,y_bg = np.histogram2d(X_bg,Y_bg, bins = 501)


#Smooth with filter
ext = (np.min(X), np.max(X), np.min(Y), np.max(Y))
im = plt.imshow(data.T, interpolation = 'gaussian', cmap = cmap, norm=norm, extent = ext, animated=True)
ax.set_xlabel('R [kpc]')
ax.set_ylabel('z [kpc]')
time_text = ax.text(0.1, 0.9,'', color = 'orange', transform=ax.transAxes, fontsize=16)

#Define animation. 
def animate(i) :
    
    X = R_mergers[i]
    Y = z_mergers[i]
    X = np.append(X, [-201, 201])
    Y = np.append(Y, [-201, 201])
    data,x,y = np.histogram2d(X,Y, bins = 501)
    im.set_data(data.T)
    time_text.set_text('Snapshot ' + str(snapnrs[i]) )
    

anim_Rz = animation.FuncAnimation(fig, animate, frames=len(s_snap))

plt.show()

anim_Rz.save('../plots/merger73_spatial_Rz_dist.gif',writer='imagemagick',fps=3)



In [ ]:
cmap = copy.copy(plt.cm.inferno)
cmap.set_bad((0,0,0))  # Fill background with black

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
h = ax.hist2d(rxy2_merg_kpc, z2_merg_kpc, bins=401, norm=matplotlib.colors.LogNorm(), cmap = cmap)
ax.set_xlabel("R [kpc]")
ax.set_ylabel("z [kpc]")
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.2)
fig.colorbar(h[3], cax=cax)


In [ ]:
pot_galpy = setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB)
for ID in id_list:    
    gcmask = np.isin(s.id, ID)

    if np.sum(gcmask) == 0:
        continue
    # get position and velocities of all selected GCs & convert to galpy units
    (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, gcmask)
    R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / _REFR0_kpc, vR_kms / _REFV0_kms, vphi_kms / _REFV0_kms, z_kpc / _REFR0_kpc, vz_kms / _REFV0_kms

    # estimate Delta of the Staeckel potential
    delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)

    # set up the actionAngleStaeckel object
    aAS = actionAngleStaeckel(
            pot   = pot_galpy,  # potential
            delta = delta,      # focal length of confocal coordinate system
            c     = True        # use C code (for speed)
            )
    # calculate actions (in galpy units) & convert them to physical units
    r_condition = np.where((np.sqrt(R_kpc**2 + z_kpc**2)) <= 50.)

    jR_galpy, lz_galpy, jz_galpy = aAS(R_galpy[r_condition], vR_galpy[r_condition], vT_galpy[r_condition], z_galpy[r_condition], vz_galpy[r_condition])
    jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * _REFR0_kpc * _REFV0_kms, lz_galpy * _REFR0_kpc * _REFV0_kms, jz_galpy * _REFR0_kpc * _REFV0_kms

    # prepare data for plotting and create corner plot
    data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
    labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']
    if init == 0:
        figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])
        init +=1
    else:
        figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, fig = figure, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])            
    #figure.suptitle(num + ' ' + title + '\nnumber of GCs: ' + str(len(R_kpc[r_condition])) + '\n$\Delta$ Staeckel: ' + "{:2.2f}".format(delta));

    #display(figure)
    if repeater == 0:
        print('number of all selected GCs', np.sum(gcmask), 'num of GCs within 50kpc:', len(r_condition[0]))


In [ ]:
s_snap = np.arange(startnr,endnr,1)

fig, ax = plt.subplots(figsize = (8,8))

data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']


figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, fig = figure, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])            

#Smooth with filter
ext = (np.min(X), np.max(X), np.min(Y), np.max(Y))
im = plt.imshow(data.T, interpolation = 'gaussian', cmap = cmap, norm=norm, extent = ext, animated=True)
ax.set_xlabel('x [kpc]')
ax.set_ylabel('y [kpc]')
time_text = ax.text(0.1, 0.9,'', color = 'orange', transform=ax.transAxes, fontsize=16)

#Define animation. 
def animate(i) :
    
    X = x_mergers[i]
    Y = y_mergers[i]
    X = np.append(X, [-201, 201])
    Y = np.append(Y, [-201, 201])
    data,x,y = np.histogram2d(X,Y, bins = 501)
    im.set_data(data.T)
    time_text.set_text('Snapshot ' + str(snapnrs[i]) )
    

anim_xy = animation.FuncAnimation(fig, animate, frames=len(s_snap))

plt.show()

anim_xy.save('../plots/merger73_spatial_xy_dist.gif',writer='imagemagick',fps=3)

### for Rz plot ###
X = R_mergers[0]
Y = z_mergers[0]
X = np.append(X, [-151, 151])
Y = np.append(Y, [-151, 151])

fig, ax = plt.subplots(figsize = (8,8))
ax.set_ylim([-150, 150])
ax.set_xlim([-150, 150])
#Create 2d Histogram
data,x,y = np.histogram2d(X,Y, bins = 401)
#data_bg,x_bg,y_bg = np.histogram2d(X_bg,Y_bg, bins = 501)


#Smooth with filter
ext = (np.min(X), np.max(X), np.min(Y), np.max(Y))
im = plt.imshow(data.T, interpolation = 'gaussian', cmap = cmap, norm=norm, extent = ext, animated=True)
ax.set_xlabel('R [kpc]')
ax.set_ylabel('z [kpc]')
time_text = ax.text(0.1, 0.9,'', color = 'orange', transform=ax.transAxes, fontsize=16)

#Define animation. 
def animate(i) :
    
    X = R_mergers[i]
    Y = z_mergers[i]
    X = np.append(X, [-201, 201])
    Y = np.append(Y, [-201, 201])
    data,x,y = np.histogram2d(X,Y, bins = 501)
    im.set_data(data.T)
    time_text.set_text('Snapshot ' + str(snapnrs[i]) )
    

anim_Rz = animation.FuncAnimation(fig, animate, frames=len(s_snap))

plt.show()

anim_Rz.save('../plots/merger73_spatial_Rz_dist.gif',writer='imagemagick',fps=3)



In [ ]:
# schau die animationsdokumentation genauer an um zu sehen was ich wann einlesen muss und worueber gelooped werden muss
# entweder save as files oder finde einen anderen weg, wie ich die ids kriege 
def plot_action_corner():
    

In [ ]:
#### path = /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/*

### ____ fit actions to stars so only load type 4 ____

level = 4

startnr = 72 # nr of snapshot where dwarf is first merged

j = 0
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"
    for snapnr in range(startnr,128,1):
        print("snapnr  : {0}".format(snapnr))
        print(datetime.datetime.now().time())
        try:
            s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
            haloid=0, galradfac=0.1, verbose=False) 
        except KeyError:
            print('\n\n', snapnr, 'not read in.\n\n')
            continue

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )
        '''
        if j == 0:
            # initialize figure if not given as input
            j +=1
        else:
        '''

In [ ]:
### write a function out of the routine above
# read in ID lists generated in notebook 2dwarf

#def 

org_dwarfs = ['72']
pot_fit_routines = ['diff_ev', 'emcee1', 'emcee2']
repeater = 0
for routine in pot_fit_routines:
    init = 0

    for num in org_dwarfs:
        if num == '57':
            color = 'red'
        elif num == '67':
            color = 'blue'
        elif num == '72':
            color = 'green' 
            print('Heloohohohohohoheihfhufbe')
        ids_notused = np.loadtxt(filedir + 'surviving_ids_snapshot_' + num + '_sh_1.txt', dtype = 'int') # 2648 particles 
        ids = np.loadtxt(filedir + 'surviving_ids_zsignchange_snapshot_' + num + '_sh_1.txt', dtype = 'int') # 93 particles

        id_list = [ids_notused] #ids

        # read in the different results of the different fitting routines of notebook 1pot
        
        if routine == 'diff_ev':
            v0_kms, a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB = np.loadtxt(filedir + 'pot_res_diff_ev.txt')
            title = 'differential evolution'
        elif routine == 'emcee1':
            repeater = 1
            v0_kms, a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB = np.loadtxt(filedir + 'pot_res_emcee1.txt')
            title = '1st MCMC'
        elif routine == 'emcee2':
            v0_kms, a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB = np.loadtxt(filedir + 'pot_res_emcee2.txt')
            title = '2nd MCMC'
        else:
            raise FileNotFoundError  

        _REFV0_kms = v0_kms

        pot_galpy = setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB)
        for ID in id_list:    
            gcmask = np.isin(s.id, ID)

            if np.sum(gcmask) == 0:
                continue
            # get position and velocities of all selected GCs & convert to galpy units
            (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, gcmask)
            R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / _REFR0_kpc, vR_kms / _REFV0_kms, vphi_kms / _REFV0_kms, z_kpc / _REFR0_kpc, vz_kms / _REFV0_kms

            # estimate Delta of the Staeckel potential
            delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)

            # set up the actionAngleStaeckel object
            aAS = actionAngleStaeckel(
                    pot   = pot_galpy,  # potential
                    delta = delta,      # focal length of confocal coordinate system
                    c     = True        # use C code (for speed)
                    )
            # calculate actions (in galpy units) & convert them to physical units
            r_condition = np.where((np.sqrt(R_kpc**2 + z_kpc**2)) <= 50.)
            
            jR_galpy, lz_galpy, jz_galpy = aAS(R_galpy[r_condition], vR_galpy[r_condition], vT_galpy[r_condition], z_galpy[r_condition], vz_galpy[r_condition])
            jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * _REFR0_kpc * _REFV0_kms, lz_galpy * _REFR0_kpc * _REFV0_kms, jz_galpy * _REFR0_kpc * _REFV0_kms

            # prepare data for plotting and create corner plot
            data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
            labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']
            if init == 0:
                figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])
                init +=1
            else:
                figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, fig = figure, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])            
            #figure.suptitle(num + ' ' + title + '\nnumber of GCs: ' + str(len(R_kpc[r_condition])) + '\n$\Delta$ Staeckel: ' + "{:2.2f}".format(delta));
            
            #display(figure)
            if repeater == 0:
                print('number of all selected GCs', np.sum(gcmask), 'num of GCs within 50kpc:', len(r_condition[0]))
    '''            
    #MW potential
    from galpy.potential import MWPotential
    MWV0_kms = 220

    # estimate Delta of the Staeckel potential
    delta = estimateDeltaStaeckel(MWPotential, R_galpy, z_galpy)

    # set up the actionAngleStaeckel object
    aAS = actionAngleStaeckel(
            pot   = MWPotential,  # potential
            delta = delta,      # focal length of confocal coordinate system
            c     = True        # use C code (for speed)
            )
    # calculate actions (in galpy units) & convert them to physical units
    r_condition = np.where((R_kpc + z_kpc) <= 50.)

    jR_galpy, lz_galpy, jz_galpy = aAS(R_galpy[r_condition], vR_galpy[r_condition], vT_galpy[r_condition], z_galpy[r_condition], vz_galpy[r_condition])
    jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * _REFR0_kpc * MWV0_kms, lz_galpy * _REFR0_kpc * MWV0_kms, jz_galpy * _REFR0_kpc * MWV0_kms

    data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
    labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']
    figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = 'black', fig = figure)            
    
    
    '''

In [ ]:
data = np.vstack([R_kpc[r_condition], phi_rad[r_condition], z_kpc[r_condition]])
labels = ['R[kpc]', 'phi[rad]', 'z[kpc]']
corner.corner(data.transpose(), labels = labels, plot_contours = 0, color = color)

In [ ]:
# read in ID lists generated in notebook 2dwarf
org_dwarfs = ['72']
pot_fit_routines = ['emcee1']
repeater = 0
for routine in pot_fit_routines:
    init = 0

    for num in org_dwarfs:
        if num == '57':
            color = 'red'
        elif num == '67':
            color = 'blue'
        elif num == '72':
            color = 'green'            
        ids_notused = np.loadtxt(filedir + 'surviving_ids_snapshot_' + num + '_sh_1.txt', dtype = 'int') # 2648 particles 
        ids = np.loadtxt(filedir + 'surviving_ids_zsignchange_snapshot_' + num + '_sh_1.txt', dtype = 'int') # 93 particles

        id_list = [ids_notused] #ids
        print("ID list done")
        # read in the different results of the different fitting routines of notebook 1pot
        
        if routine == 'diff_ev':
            v0_kms, a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB = np.loadtxt(filedir + 'pot_res_diff_ev.txt')
            title = 'differential evolution'
        elif routine == 'emcee1':
            repeater = 1
            v0_kms, a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB = np.loadtxt(filedir + 'pot_res_emcee1.txt')
            title = '1st MCMC'
            print('Potential parameters there')
        elif routine == 'emcee2':
            v0_kms, a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB = np.loadtxt(filedir + 'pot_res_emcee2.txt')
            title = '2nd MCMC'
        else:
            raise FileNotFoundError  

        _REFV0_kms = v0_kms

        pot_galpy = setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB)
        print('Potential is set up.')
        for ID in id_list:    
            gcmask = np.isin(s.id, ID)

            if np.sum(gcmask) == 0:
                continue
            # get position and velocities of all selected GCs & convert to galpy units
            (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, gcmask)
            R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / _REFR0_kpc, vR_kms / _REFV0_kms, vphi_kms / _REFV0_kms, z_kpc / _REFR0_kpc, vz_kms / _REFV0_kms
            print('Galpy vals gotten.')
            # estimate Delta of the Staeckel potential
            delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)

            # set up the actionAngleStaeckel object
            aAS = actionAngleStaeckel(
                    pot   = pot_galpy,  # potential
                    delta = delta,      # focal length of confocal coordinate system
                    c     = True        # use C code (for speed)
                    )
            print('Action object setup.')
            # calculate actions (in galpy units) & convert them to physical units
            r_condition = np.where((np.sqrt(R_kpc**2 + z_kpc**2)) <= 50.)
            
            jR_galpy, lz_galpy, jz_galpy = aAS(R_galpy[r_condition], vR_galpy[r_condition], vT_galpy[r_condition], z_galpy[r_condition], vz_galpy[r_condition])
            jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * _REFR0_kpc * _REFV0_kms, lz_galpy * _REFR0_kpc * _REFV0_kms, jz_galpy * _REFR0_kpc * _REFV0_kms
            print('actions calculated')
            # prepare data for plotting and create corner plot
            data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
            labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']
            if init == 0:
                print('Figure started')
                figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])
                init +=1
            else:
                figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = color, fig = figure, range =  [(0.,12500.), (-14000.,14000.),(0., 5000.)])            
            #figure.suptitle(num + ' ' + title + '\nnumber of GCs: ' + str(len(R_kpc[r_condition])) + '\n$\Delta$ Staeckel: ' + "{:2.2f}".format(delta));
            
            #display(figure)
            if repeater == 0:
                print('number of all selected GCs', np.sum(gcmask), 'num of GCs within 50kpc:', len(r_condition[0]))
    '''            
    #MW potential
    from galpy.potential import MWPotential
    MWV0_kms = 220

    # estimate Delta of the Staeckel potential
    delta = estimateDeltaStaeckel(MWPotential, R_galpy, z_galpy)

    # set up the actionAngleStaeckel object
    aAS = actionAngleStaeckel(
            pot   = MWPotential,  # potential
            delta = delta,      # focal length of confocal coordinate system
            c     = True        # use C code (for speed)
            )
    # calculate actions (in galpy units) & convert them to physical units
    r_condition = np.where((R_kpc + z_kpc) <= 50.)

    jR_galpy, lz_galpy, jz_galpy = aAS(R_galpy[r_condition], vR_galpy[r_condition], vT_galpy[r_condition], z_galpy[r_condition], vz_galpy[r_condition])
    jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * _REFR0_kpc * MWV0_kms, lz_galpy * _REFR0_kpc * MWV0_kms, jz_galpy * _REFR0_kpc * MWV0_kms

    data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
    labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']
    figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = 'black', fig = figure)            
    
    
    '''